New approach:
- A dataset of 1000 data points from one of the sensors per each walking surface from each user (1-30).
- Omit Butterworth filter for this notebook

In [20]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

In [2]:
data = pd.read_csv("sampled_data/data.csv")
data.describe()

,UserId,PacketCounter,SampleTimeFine,Acc_X,Acc_Y,Acc_Z,FreeAcc_X,FreeAcc_Y,FreeAcc_Z,Gyr_X,...,OriInc_q3,Roll,Pitch,Yaw,Latitude,Longitude,Altitude,Vel_X,Vel_Y,Vel_Z
count,1.708236e+06,1.708236e+06,0.0,1.693804e+06,1.693804e+06,1.693804e+06,1.693804e+06,1.693804e+06,1.693804e+06,1.693804e+06,...,1.693804e+06,1.693804e+06,1.693804e+06,1.693804e+06,0.0,0.0,1708236.0,0.0,0.0,0.0
mean,1.550002e+01,3.225632e+04,NaN,9.838823e+00,9.067474e-01,-7.830717e-02,-1.514832e-01,1.432863e-01,-1.293384e-01,4.703301e-02,...,-6.346451e-04,3.895021e+01,-7.090170e+01,-6.138306e-01,NaN,NaN,0.0,NaN,NaN,NaN
std,8.654662e+00,1.917977e+04,NaN,1.490878e+02,2.010112e+02,8.939607e+01,1.354799e+02,1.363478e+02,1.126015e+02,2.040233e+00,...,3.301078e-03,9.922891e+01,9.823791e+00,1.003020e+02,NaN,NaN,0.0,NaN,NaN,NaN
min,1.000000e+00,0.000000e+00,NaN,-1.376947e+05,-1.854641e+05,-9.034852e+01,-1.246128e+05,-7.910270e+01,-1.034285e+05,-1.959829e+02,...,-7.032000e-02,-1.800000e+02,-8.999649e+01,-1.799997e+02,NaN,NaN,0.0,NaN,NaN,NaN
25%,8.000000e+00,1.507900e+04,NaN,7.076802e+00,-8.746855e-01,-2.809921e+00,-2.008377e+00,-2.668110e+00,-2.727015e+00,-1.031616e+00,...,-2.157000e-03,-3.304743e+00,-7.826868e+01,-7.961625e+01,NaN,NaN,0.0,NaN,NaN,NaN
50%,1.600000e+01,3.216300e+04,NaN,9.597096e+00,8.244235e-01,-7.924415e-01,-4.086850e-02,2.297000e-03,-7.947700e-02,4.664750e-02,...,-2.370000e-04,2.974054e+01,-7.164629e+01,-3.291615e+01,NaN,NaN,0.0,NaN,NaN,NaN
75%,2.300000e+01,4.921100e+04,NaN,1.306224e+01,2.870030e+00,1.985747e+00,1.995095e+00,2.776245e+00,2.630229e+00,1.115163e+00,...,1.069000e-03,1.306391e+02,-6.512706e+01,8.225846e+01,NaN,NaN,0.0,NaN,NaN,NaN
max,3.000000e+01,6.553500e+04,NaN,1.194948e+02,9.340336e+01,8.577615e+04,6.455670e+01,1.253817e+05,1.569266e+02,2.000270e+02,...,5.071500e-02,1.799999e+02,7.346872e+01,1.799999e+02,NaN,NaN,0.0,NaN,NaN,NaN


In [3]:
data

,UserId,Surface,SensorLoc,PacketCounter,SampleTimeFine,Acc_X,Acc_Y,Acc_Z,FreeAcc_X,FreeAcc_Y,...,OriInc_q3,Roll,Pitch,Yaw,Latitude,Longitude,Altitude,Vel_X,Vel_Y,Vel_Z
0,1,CALIB,Left thigh,31570,NaN,9.492260,0.130429,-2.558672,-0.000000,-0.000000,...,0.000024,177.082561,-74.903636,103.717809,NaN,NaN,0.0,NaN,NaN,NaN
1,1,CALIB,Left thigh,31571,NaN,9.521406,0.118498,-2.408461,-0.046313,0.186881,...,0.000043,177.231488,-74.656365,104.388684,NaN,NaN,0.0,NaN,NaN,NaN
2,1,CALIB,Left thigh,31572,NaN,9.544091,0.187095,-2.204060,-0.019889,0.402747,...,0.000036,177.248779,-74.640533,104.370032,NaN,NaN,0.0,NaN,NaN,NaN
3,1,CALIB,Left thigh,31573,NaN,9.573117,0.162900,-2.168100,-0.052578,0.442471,...,0.000009,177.257435,-74.619164,104.360623,NaN,NaN,0.0,NaN,NaN,NaN
4,1,CALIB,Left thigh,31574,NaN,9.653494,0.251947,-2.118363,0.018708,0.531586,...,-0.000034,177.242447,-74.599610,104.371823,NaN,NaN,0.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1708231,30,BnkR,Left thigh,16965,NaN,9.495196,2.108930,-0.187773,-0.218522,1.552237,...,0.002268,142.668222,-77.360093,166.120602,NaN,NaN,0.0,NaN,NaN,NaN
1708232,30,BnkR,Left thigh,16966,NaN,9.391318,1.741557,0.061527,-0.761521,1.758386,...,0.002806,148.320584,-76.308703,161.225308,NaN,NaN,0.0,NaN,NaN,NaN
1708233,30,BnkR,Left thigh,16967,NaN,9.193044,1.086965,0.608281,-1.664782,1.969962,...,0.003157,153.169073,-75.238857,157.083508,NaN,NaN,0.0,NaN,NaN,NaN
1708234,30,BnkR,Left thigh,16968,NaN,9.350600,0.579385,0.986258,-2.418844,2.146061,...,0.003333,157.235699,-74.179888,153.625064,NaN,NaN,0.0,NaN,NaN,NaN


In [4]:
print("User ids:", " ".join(map(str, data.UserId.unique())))
print("Surfaces:", " ".join(map(str, data.Surface.unique())))
print("Sensor locations:", " ".join(map(str, data.SensorLoc.unique())))

User ids: 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30
Surfaces: CALIB SlpD FE SlpU BnkL BnkR GR StrU CS StrD
Sensor locations: Left thigh


In [5]:
# Drop unneeded and NaN columns
data.dropna(axis=1, how="all", inplace=True)
data.drop(["PacketCounter", "Altitude", "SensorLoc"], axis=1, inplace=True)
data.head(5)

,UserId,Surface,Acc_X,Acc_Y,Acc_Z,FreeAcc_X,FreeAcc_Y,FreeAcc_Z,Gyr_X,Gyr_Y,...,VelInc_X,VelInc_Y,VelInc_Z,OriInc_q0,OriInc_q1,OriInc_q2,OriInc_q3,Roll,Pitch,Yaw
0,1,CALIB,9.492260,0.130429,-2.558672,-0.000000,-0.000000,0.019241,-0.025877,-0.028406,...,0.094926,0.001303,-0.025573,1.0,-0.000129,-0.000142,0.000024,177.082561,-74.903636,103.717809
1,1,CALIB,9.521406,0.118498,-2.408461,-0.046313,0.186881,0.007437,-0.002117,-0.030708,...,0.095218,0.001189,-0.024070,1.0,-0.000011,-0.000154,0.000043,177.231488,-74.656365,104.388684
2,1,CALIB,9.544091,0.187095,-2.204060,-0.019889,0.402747,-0.023921,-0.001215,-0.028015,...,0.095444,0.001874,-0.022027,1.0,-0.000006,-0.000140,0.000036,177.248779,-74.640533,104.370032
3,1,CALIB,9.573117,0.162900,-2.168100,-0.052578,0.442471,-0.005893,-0.000991,-0.038044,...,0.095735,0.001630,-0.021663,1.0,-0.000005,-0.000190,0.000009,177.257435,-74.619164,104.360623
4,1,CALIB,9.653494,0.251947,-2.118363,0.018708,0.531586,0.059392,-0.007314,-0.033840,...,0.096539,0.002515,-0.021167,1.0,-0.000037,-0.000169,-0.000034,177.242447,-74.599610,104.371823


In [6]:
from sklearn import preprocessing

# Encode data
le = preprocessing.LabelEncoder()
data["Surface"] = le.fit_transform(data["Surface"])

### Partition data in the following way:
User Id | Surface\
1       | 1\
2       | 1\
3       | 1\
4       | 2\
5       | 2\
6       | 3\
7       | 3

This way we'll have distinct partitions of measurements per each class (surface), which would be similar to what was done in [TSFresh example](https://github.com/blue-yonder/tsfresh/blob/main/notebooks/examples/04%20Multiclass%20Selection%20Example.ipynb)


In [7]:
# Groups
groups = (
    # User id, surface
    (1, 1),
    (2, 1),
    (3, 1),
    (4, 2),
    (5, 2),
    (6, 3),
    (7, 3)
)

datasets = []
for user_id, surface in groups:
    datasets.append(data[(data.UserId == user_id) & (data.Surface == surface)].head(100))

data_ts = pd.concat(datasets)
data_ts

,UserId,Surface,Acc_X,Acc_Y,Acc_Z,FreeAcc_X,FreeAcc_Y,FreeAcc_Z,Gyr_X,Gyr_Y,...,VelInc_X,VelInc_Y,VelInc_Z,OriInc_q0,OriInc_q1,OriInc_q2,OriInc_q3,Roll,Pitch,Yaw
7000,1,1,9.655841,-0.802532,-2.329289,0.000000,0.000000,0.152498,-0.091646,-0.017493,...,0.096563,-0.008009,-0.023281,1.000000,-0.000458,-0.000087,0.000278,-161.015603,-75.696432,106.508134
7001,1,1,9.666487,-0.702016,-2.169317,-0.000000,0.000000,0.119066,-0.067123,-0.004504,...,0.096667,-0.007007,-0.021689,1.000000,-0.000336,-0.000023,0.000211,-162.095639,-76.732910,105.582259
7002,1,1,9.825154,-0.555551,-1.946949,-0.017393,0.384734,0.211513,-0.083571,0.017835,...,0.098251,-0.005550,-0.019476,1.000000,-0.000418,0.000089,0.000135,-161.539449,-76.228889,105.926404
7003,1,1,9.658233,-0.667679,-1.916895,-0.113715,0.306461,0.051130,-0.037595,0.045210,...,0.096579,-0.006672,-0.019190,1.000000,-0.000188,0.000226,0.000084,-161.490286,-76.250413,105.853612
7004,1,1,9.604000,-0.419122,-2.130551,0.217647,0.257760,0.027938,-0.020632,0.042873,...,0.096036,-0.004187,-0.021326,1.000000,-0.000103,0.000214,0.000068,-161.444812,-76.275139,105.796783
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
352813,7,3,32.040164,8.295128,25.252114,-2.505447,7.876112,30.988274,0.127529,0.851674,...,0.321848,0.081319,0.251205,0.999980,0.000638,0.004258,-0.004579,16.758287,-61.421788,-66.939328
352814,7,3,15.722644,3.429713,20.094727,-6.167401,8.344551,13.747471,-1.350992,1.247159,...,0.158614,0.035016,0.199726,0.999950,-0.006755,0.006236,-0.004006,16.411058,-60.606363,-67.427329
352815,7,3,3.320524,-3.991464,-3.771406,-0.368725,-6.381341,-9.244025,-1.006403,-0.193677,...,0.033230,-0.040113,-0.037480,0.999987,-0.005032,-0.000968,-0.000280,15.832500,-60.620537,-67.471974
352816,7,3,-2.057958,-7.734008,-11.954224,0.310984,-11.637539,-18.263975,0.131341,-0.292732,...,-0.020323,-0.077283,-0.119623,0.999998,0.000657,-0.001464,0.001055,15.782609,-60.815291,-67.328544


In [8]:
# Add Time column
data_ts.insert(1, "Time", list(range(1, len(data_ts) + 1)))
data_ts

,UserId,Time,Surface,Acc_X,Acc_Y,Acc_Z,FreeAcc_X,FreeAcc_Y,FreeAcc_Z,Gyr_X,...,VelInc_X,VelInc_Y,VelInc_Z,OriInc_q0,OriInc_q1,OriInc_q2,OriInc_q3,Roll,Pitch,Yaw
7000,1,1,1,9.655841,-0.802532,-2.329289,0.000000,0.000000,0.152498,-0.091646,...,0.096563,-0.008009,-0.023281,1.000000,-0.000458,-0.000087,0.000278,-161.015603,-75.696432,106.508134
7001,1,2,1,9.666487,-0.702016,-2.169317,-0.000000,0.000000,0.119066,-0.067123,...,0.096667,-0.007007,-0.021689,1.000000,-0.000336,-0.000023,0.000211,-162.095639,-76.732910,105.582259
7002,1,3,1,9.825154,-0.555551,-1.946949,-0.017393,0.384734,0.211513,-0.083571,...,0.098251,-0.005550,-0.019476,1.000000,-0.000418,0.000089,0.000135,-161.539449,-76.228889,105.926404
7003,1,4,1,9.658233,-0.667679,-1.916895,-0.113715,0.306461,0.051130,-0.037595,...,0.096579,-0.006672,-0.019190,1.000000,-0.000188,0.000226,0.000084,-161.490286,-76.250413,105.853612
7004,1,5,1,9.604000,-0.419122,-2.130551,0.217647,0.257760,0.027938,-0.020632,...,0.096036,-0.004187,-0.021326,1.000000,-0.000103,0.000214,0.000068,-161.444812,-76.275139,105.796783
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
352813,7,696,3,32.040164,8.295128,25.252114,-2.505447,7.876112,30.988274,0.127529,...,0.321848,0.081319,0.251205,0.999980,0.000638,0.004258,-0.004579,16.758287,-61.421788,-66.939328
352814,7,697,3,15.722644,3.429713,20.094727,-6.167401,8.344551,13.747471,-1.350992,...,0.158614,0.035016,0.199726,0.999950,-0.006755,0.006236,-0.004006,16.411058,-60.606363,-67.427329
352815,7,698,3,3.320524,-3.991464,-3.771406,-0.368725,-6.381341,-9.244025,-1.006403,...,0.033230,-0.040113,-0.037480,0.999987,-0.005032,-0.000968,-0.000280,15.832500,-60.620537,-67.471974
352816,7,699,3,-2.057958,-7.734008,-11.954224,0.310984,-11.637539,-18.263975,0.131341,...,-0.020323,-0.077283,-0.119623,0.999998,0.000657,-0.001464,0.001055,15.782609,-60.815291,-67.328544


In [22]:
from tsfresh import extract_features, select_features
# from tsfresh.feature_extraction.settings import MinimalFCParameter
from tsfresh.utilities.dataframe_functions import impute

In [23]:
fts = extract_features(data_ts, column_id="UserId", column_sort="Time", impute_function=impute)
fts

Feature Extraction: 100%|██████████| 27/27 [00:05<00:00,  4.53it/s]


,Surface__variance_larger_than_standard_deviation,Surface__has_duplicate_max,Surface__has_duplicate_min,Surface__has_duplicate,Surface__sum_values,Surface__abs_energy,Surface__mean_abs_change,Surface__mean_change,Surface__mean_second_derivative_central,Surface__median,...,Yaw__permutation_entropy__dimension_6__tau_1,Yaw__permutation_entropy__dimension_7__tau_1,Yaw__query_similarity_count__query_None__threshold_0.0,"Yaw__matrix_profile__feature_""min""__threshold_0.98","Yaw__matrix_profile__feature_""max""__threshold_0.98","Yaw__matrix_profile__feature_""mean""__threshold_0.98","Yaw__matrix_profile__feature_""median""__threshold_0.98","Yaw__matrix_profile__feature_""25""__threshold_0.98","Yaw__matrix_profile__feature_""75""__threshold_0.98",Yaw__mean_n_absolute_max__number_of_maxima_7
1,0.0,1.0,1.0,1.0,100.0,100.0,0.0,0.0,0.0,1.0,...,1.845727,2.128202,0.0,1.471482,4.636555,3.108006,3.133374,2.513651,3.937026,105.870555
2,0.0,1.0,1.0,1.0,100.0,100.0,0.0,0.0,0.0,1.0,...,3.149006,3.349272,0.0,0.898623,4.881833,1.987091,1.854978,1.334374,2.419331,83.131141
3,0.0,1.0,1.0,1.0,100.0,100.0,0.0,0.0,0.0,1.0,...,1.670518,1.827991,0.0,1.267620,6.519683,3.524110,3.143929,2.903679,4.416077,153.319153
4,0.0,1.0,1.0,1.0,200.0,400.0,0.0,0.0,0.0,2.0,...,2.380491,2.581217,0.0,1.453640,10.983563,4.123224,2.730005,1.931807,4.450049,170.183915
5,0.0,1.0,1.0,1.0,200.0,400.0,0.0,0.0,0.0,2.0,...,1.709673,1.958530,0.0,1.337857,11.082836,3.964209,2.622923,1.753662,4.031431,172.376181
6,0.0,1.0,1.0,1.0,300.0,900.0,0.0,0.0,0.0,3.0,...,1.070237,1.169149,0.0,1.451520,7.081264,3.627020,3.836201,1.999783,5.430348,179.256418
7,0.0,1.0,1.0,1.0,300.0,900.0,0.0,0.0,0.0,3.0,...,1.191335,1.289129,0.0,1.563953,8.118662,3.625416,3.966219,2.566578,4.296657,177.810574


In [24]:
user_surfaces = data_ts[["UserId", "Surface"]].groupby(["UserId", "Surface"]).count().reset_index()
targets = pd.Series(user_surfaces["Surface"].values, user_surfaces["UserId"].values)
targets

1    1
2    1
3    1
4    2
5    2
6    3
7    3
dtype: int64

In [25]:
relevant_features = set()

for label in targets.unique():
    targets_binary = targets == label
    fts_filtered = select_features(fts, targets_binary)
    print("Number of relevant features for class {}: {}/{}".format(label, fts_filtered.shape[1], fts.shape[1]))
    relevant_features = relevant_features.union(set(fts_filtered.columns))

Number of relevant features for class 1: 0/18147
Number of relevant features for class 2: 0/18147
Number of relevant features for class 3: 0/18147


Result: no features found useful by TSFresh